In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import time
from pprint import pprint
import pandas as pd
from selenium.webdriver.chrome.options import Options
import requests
import json
from config import geoapify_key
import re

browser = Browser('chrome')

# # Chrome options and preferences
# chrome_options = Options()
# chrome_options.add_argument("--blink-settings=imagesEnabled=false")

# # Chrome with options
# browser = Browser('chrome', options=chrome_options)


In [2]:

# Load the web page
url = "https://www.lcbo.com/en/products#t=clp-products&sort=relevancy&layout=card&f:@lcbo_current_offer=[On%20Sale]&f:@ec_category=[Wine]"
browser.visit(url)

# To select "English" as preffered language
time.sleep(3)
results = browser.links.find_by_text('English')
results.click()
time.sleep(3)

html = browser.html
soup=BeautifulSoup(html, 'html.parser')

In [3]:
# Removes the cookie popup
cookie = browser.find_by_id('btn-cookie-allow').click()

In [4]:
iteration = 0
while True:
    try:     
        # Wait for 1 second
        time.sleep(1)
        # Click on Load More button
        browser.find_by_id('loadMore').click()
    except:
        break
    print(iteration)
    iteration += 1

0
1
2


In [5]:
html = browser.html
    
soup = BeautifulSoup(html, 'html.parser')

all_links = soup.find_all("div", class_ = 'coveo-product-items')

links = []

for link in all_links:
    results = link.a['href']
    links.append(results)
        

In [6]:
len(links)
print(links)

['https://www.lcbo.com/en/collavini-pinot-grigio-33340', 'https://www.lcbo.com/en/ruffino-prosecco-rose-doc-20896', 'https://www.lcbo.com/en/josh-cellars-pinot-grigio-15754', 'https://www.lcbo.com/en/louis-m-martini-sonoma-county-cabernet-sauvignon-292151', 'https://www.lcbo.com/en/ogier-cotes-du-ventoux-rose-aoc-134916', 'https://www.lcbo.com/en/ruffino-chianti-1743', 'https://www.lcbo.com/en/oyster-bay-sauvignon-blanc-316570', 'https://www.lcbo.com/en/19-crimes-snoop-dogg-cali-red-18116', 'https://www.lcbo.com/en/peller-family-vineyards-pinot-grigio-667089', 'https://www.lcbo.com/en/monte-zovo-sa-solin-ripasso-valpolicella-650713', 'https://www.lcbo.com/en/liberty-school-cabernet-sauvignon-738823', 'https://www.lcbo.com/en/babich-sauvignon-blanc-620054', 'https://www.lcbo.com/en/ruffino-lumina-pinot-grigio-igt-589101', 'https://www.lcbo.com/en/casal-garcia-vinho-verde-do-530261', 'https://www.lcbo.com/en/bottega-vino-dei-poeti-brut-rose-277202', 'https://www.lcbo.com/en/barone-ricaso

In [7]:
regular_price = []
sale_price = []
name = []
rating = []
onsale = [] #this is the On Sale tag
reviews = []
size = []
description = []
category = []
details_list = []
taste_list = []


max_iteration2 = 5
iteration2 = 0


for record in links:
    if iteration2 >= max_iteration2:
        break
    url = record
    browser.visit(url)
    time.sleep(2)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
#     time.sleep(1)
#     browser.find_by_css('my_store_close_button').click()
    
    try:
        old_price = soup.find('span', class_ = 'old-price')
        old = old_price.find('span', class_ = 'price')
        regular_price.append(old.find('span', class_ = 'price').text)
    except AttributeError:
        regular_price.append('N/A')

    try:
        on_sale_price = soup.find('span', class_ = 'special-price')
        new_price = on_sale_price.find('span', class_ = 'price')
        sale_price.append(new_price.find('span', class_ = 'price').text)
    except AttributeError:
        sale_price.append('N/A')
    
    try:    
        name.append(soup.find('div', class_ = 'page-title-wrapper').text)
    except AttributeError:
        name.append('N/A')
        
    try:
        onsale.append(soup.find('div', class_ = 'amlabel-text').text)
    except AttributeError:
        onsale.append('N/A')
    
    try:
        rating.append(soup.find('div', class_ = 'bv_avgRating_component_container notranslate').text)
    except AttributeError:
        rating.append('N/A')
    
    try:
        reviews.append(soup.find('div', class_ = 'bv_numReviews_text').text)
    except AttributeError:
        reviews.append('N/A')
    
    try:
        size.append(soup.find('div', class_ = 'lcbo-product-size').text)
    except AttributeError:
        size.append('N/A')
        
    try:
        description.append(soup.find('div', class_ = 'testing_note').text)
    except AttributeError:
        description.append('N/A')
        
    categories_all = soup.find('div', class_ = 'breadcrumbs')
    categories = categories_all.find_all('li', class_ = 'item')
    category_list = []

    for category_record in categories:
        category_list.append(category_record.text)
    category.append(category_list[-2])

    moredetail = soup.find('div', class_ = 'moredetail')
    all_details = moredetail.find_all('li')
    details = {}

    for detail_record in all_details:
        details_value = detail_record.find('div', class_ = 'value').text
        details_name = detail_record.find('div', class_ = 'label').text
        details[details_name] = details_value

    details_list.append(details)
    
#     try:
#         flavour_all = soup.find('div', class_ = 'foodParings pip-info')
#         flavour = flavour_all.find('div', class_ = 'label').text
#         flavour_value = flavour_all.find('div', class_ = 'value').text
#         flavours = {
#             flavour: flavour_value
#         }
#     except AttributeError:
#         flavours = {
#             'Flavour': 'N/A'
#         }
        
    
#     try:
#         all_taste = soup.find('div', class_ = 'foodParings pip-info')
#         li_all = all_taste.find_all('li')

#         for li in li_all:
#             label = li.find('div', class_='label').text.strip()
#             value_element = li.find('div', class_='value')
#             if 'aria-label' in value_element.attrs:
#                 aria_label = value_element['aria-label']
#                 number = re.findall(r'\d+', aria_label)
#                 if number:
#                     taste_list.append({
#                         label: int(number[0])
#                     })
#     except AttributeError:
#         taste_list.append('N/A')
        
    print(iteration2)
    iteration2 += 1



0
1
2
3
4


In [8]:
data = {
    'Name': name,
    'Regular Price': regular_price,
    'Sale Price': sale_price,
    'Rating': rating,
    'Reviews': reviews,
    'Size': size,
    'Description': description,
    'Category': category,
    'Details': details_list,
#     'Flavours': flavours
    
}

# Add the details dictionary to the data dictionary
data['Details'] = details_list
# data['Taste'] = taste_list

# Create a dataframe from the data dictionary
df = pd.DataFrame(data)

details_df = pd.json_normalize(df['Details'])
# taste_df = pd.json_normalize(df['Taste'])

df = pd.concat([df, details_df], axis = 1)
# df2 = pd.concat([df1, taste_df], axis = 1)

df.drop('Details', axis = 1, inplace = True)
# df2.drop('Taste', axis = 1, inplace = True)

#Adding Latitude and Longitude columns to the Data Frame
df['Lat'] = ""
df['Lon'] = ""   

In [9]:
print(len(regular_price))
print(len(sale_price))
print(len(name))
print(len(rating))
print(len(onsale))
print(len(reviews))
print(len(size))
print(len(description))
print(len(category))
print(len(details_list))
# print(len(flavours))
# print(flavours)

5
5
5
5
5
5
5
5
5
5


In [10]:
#API call to get Latitude and Longitude
params = {
    "apiKey":geoapify_key,
    "format":"json"
}

# Set the base URL
base_url = "https://api.geoapify.com/v1/geocode/search"

df['Lat'] = None
df['Lon'] = None

for index, country in df['Made In'].to_frame().iterrows():
    location = country['Made In']
    params["text"] = location
    
    response = requests.get(base_url, params=params).json()

    latitude = response["results"][0]["lat"]
    longitude = response["results"][0]["lon"]
    df.loc[index, "Lat"] = latitude
    df.loc[index, "Lon"] = longitude

df.head()


,Name,Regular Price,Sale Price,Rating,Reviews,Size,Description,Category,Alcohol/Vol,Made In,By,Sugar Content,Varietal,Release Date,Lat,Lon
0,Collavini Pinot Grigio,$16.55,$14.55,4.5,(6),750 ml bottle,"This pinot grigio is a customer favourite, and...",White Wine,12.5%,"Veneto, Italy",Collavini,4 g/L,Pinot Grigio,NaN,45.647666,11.866525
1,Ruffino Prosecco Rose DOC,$20.00,$18.00,5.0,(2),750 ml bottle,Ruffino has been at the forefront of Italian w...,Rosé & Red Sparkling Wine,11%,"Friuli, Italy",Ruffino Srl Wines,21 g/L,Sparkling,NaN,45.4413,11.59666
2,Josh Cellars Pinot Grigio,$19.95,$17.95,4.5,(17),750 ml bottle,Josh Cellars is a renowned brand created to ce...,White Wine,12.5%,"California, United States",Josh Cellars,3 g/L,Pinot Grigio,NaN,36.701463,-118.755997
3,Louis M. Martini Sonoma County Cabernet Sauvi...,$25.95,$23.95,4.1,(27),750 ml bottle,This is a wine that hangs its hat on providing...,Red Wine,14%,"California, United States",Louis M. Martini,4 g/L,Cabernet Sauvignon,"April 1, 2021",36.701463,-118.755997
4,Ogier Cotes Du Ventoux Rose AOC,$15.85,$13.85,4.4,(7),750 ml bottle,From the south of the Rhône valley comes this ...,Rose Wine,12.5%,"Rhône, France",Ogier,2 g/L,Rosé,NaN,45.880235,4.564534


In [11]:
df['Regular Price'] = df['Regular Price'].str.replace('$','')
df['Sale Price'] = df['Sale Price'].str.replace('$','')

df['Savings'] = df['Regular Price'].astype(float) - df['Sale Price'].astype(float)
df.head()

C:\Users\josed\AppData\Local\Temp\ipykernel_13968\2892017696.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Regular Price'] = df['Regular Price'].str.replace('$','')
C:\Users\josed\AppData\Local\Temp\ipykernel_13968\2892017696.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Sale Price'] = df['Sale Price'].str.replace('$','')


,Name,Regular Price,Sale Price,Rating,Reviews,Size,Description,Category,Alcohol/Vol,Made In,By,Sugar Content,Varietal,Release Date,Lat,Lon,Savings
0,Collavini Pinot Grigio,16.55,14.55,4.5,(6),750 ml bottle,"This pinot grigio is a customer favourite, and...",White Wine,12.5%,"Veneto, Italy",Collavini,4 g/L,Pinot Grigio,NaN,45.647666,11.866525,2.0
1,Ruffino Prosecco Rose DOC,20.00,18.00,5.0,(2),750 ml bottle,Ruffino has been at the forefront of Italian w...,Rosé & Red Sparkling Wine,11%,"Friuli, Italy",Ruffino Srl Wines,21 g/L,Sparkling,NaN,45.4413,11.59666,2.0
2,Josh Cellars Pinot Grigio,19.95,17.95,4.5,(17),750 ml bottle,Josh Cellars is a renowned brand created to ce...,White Wine,12.5%,"California, United States",Josh Cellars,3 g/L,Pinot Grigio,NaN,36.701463,-118.755997,2.0
3,Louis M. Martini Sonoma County Cabernet Sauvi...,25.95,23.95,4.1,(27),750 ml bottle,This is a wine that hangs its hat on providing...,Red Wine,14%,"California, United States",Louis M. Martini,4 g/L,Cabernet Sauvignon,"April 1, 2021",36.701463,-118.755997,2.0
4,Ogier Cotes Du Ventoux Rose AOC,15.85,13.85,4.4,(7),750 ml bottle,From the south of the Rhône valley comes this ...,Rose Wine,12.5%,"Rhône, France",Ogier,2 g/L,Rosé,NaN,45.880235,4.564534,2.0


In [12]:
df.rename(columns={'Sugar Content' : 'Sugar Content g/L'})

,Name,Regular Price,Sale Price,Rating,Reviews,Size,Description,Category,Alcohol/Vol,Made In,By,Sugar Content g/L,Varietal,Release Date,Lat,Lon,Savings
0,Collavini Pinot Grigio,16.55,14.55,4.5,(6),750 ml bottle,"This pinot grigio is a customer favourite, and...",White Wine,12.5%,"Veneto, Italy",Collavini,4 g/L,Pinot Grigio,NaN,45.647666,11.866525,2.0
1,Ruffino Prosecco Rose DOC,20.00,18.00,5.0,(2),750 ml bottle,Ruffino has been at the forefront of Italian w...,Rosé & Red Sparkling Wine,11%,"Friuli, Italy",Ruffino Srl Wines,21 g/L,Sparkling,NaN,45.4413,11.59666,2.0
2,Josh Cellars Pinot Grigio,19.95,17.95,4.5,(17),750 ml bottle,Josh Cellars is a renowned brand created to ce...,White Wine,12.5%,"California, United States",Josh Cellars,3 g/L,Pinot Grigio,NaN,36.701463,-118.755997,2.0
3,Louis M. Martini Sonoma County Cabernet Sauvi...,25.95,23.95,4.1,(27),750 ml bottle,This is a wine that hangs its hat on providing...,Red Wine,14%,"California, United States",Louis M. Martini,4 g/L,Cabernet Sauvignon,"April 1, 2021",36.701463,-118.755997,2.0
4,Ogier Cotes Du Ventoux Rose AOC,15.85,13.85,4.4,(7),750 ml bottle,From the south of the Rhône valley comes this ...,Rose Wine,12.5%,"Rhône, France",Ogier,2 g/L,Rosé,NaN,45.880235,4.564534,2.0


In [21]:
column_lists = {}

# Iterate over the columns and extract values into lists
for column in df.columns:
    column_lists[column] = df[column].tolist()

# Access the lists for each column

name_list = column_lists['Name']
regular_price_list = column_lists['Regular Price']
sale_price_list = column_lists['Sale Price']
rating_list = column_lists['Rating']
reviews_list = column_lists['Reviews']
size_list = column_lists['Size']
description_list = column_lists['Description']
category_list = column_lists['Category']
alcohol_list = column_lists['Alcohol/Vol']
madein_list = column_lists['Made In']
by_list = column_lists['By']
varietal_list = column_lists['Varietal']
lat_list = column_lists['Lat']
lon_list = column_lists['Lon']
savings_list = column_lists['Savings']

data2 = {
    'name': name_list,
    'regular_price': regular_price_list,
    'sale_price': sale_price_list,
    'rating': rating_list,
    'reviews': reviews_list,
    'size': size_list,
    'description': description_list,
    'category': category_list,
    'alcohol_vol': alcohol_list,
    'made_in': madein_list,
    'by': by_list,
    'varietal': varietal_list,
    'lat': lat_list,
    'lon': lon_list,
    'savings': savings_list
}

with open('data3.json', 'w') as file:
    json.dump(data2, file)

In [19]:
print(data2)

{'name': [' Collavini Pinot Grigio', ' Ruffino Prosecco Rose DOC', ' Josh Cellars Pinot Grigio', ' Louis M. Martini Sonoma County Cabernet Sauvignon', ' Ogier Cotes Du Ventoux Rose AOC'], 'regular_price': ['16.55', '20.00', '19.95', '25.95', '15.85'], 'sale_price': ['14.55', '18.00', '17.95', '23.95', '13.85'], 'rating': ['4.5', '5.0', '4.5', '4.1', '4.4'], 'Reviews': ['(6)', '(2)', '(17)', '(27)', '(7)'], 'Size': ['  750 ml bottle ', '  750 ml bottle ', '  750 ml bottle ', '  750 ml bottle ', '  750 ml bottle '], 'Description': ["This pinot grigio is a customer favourite, and it's easy to see why. It's pale straw, with inviting aromas of lemon, pear, and apple. The palate offers flavours of apricot, grapefruit and a hint of honey. It's light-bodied and crisp, with a palate-cleansing finish. Pair with mushroom risotto or grilled shrimp.", 'Ruffino has been at the forefront of Italian winemaking since 1877. This lovely prosecco rosé is light pink and expresses approachable notes of stra

In [15]:
# df.to_csv('lcbo_wines.csv', index = False)